In [3]:
#imports
import json
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import re

In [4]:
#Read a JSON file
def read_input(file):
    content = None
    with open(file, "r") as arq:
        content = json.load(arq)
    return content

def import_floyd_warshall(filename):
    graph = nx.Graph()
    with open(filename) as file:
        data = json.load(file)
    graph.add_edges_from([(int(re.findall("(?<!\d)\d+(?!\d)",x)[0]), int(re.findall("(?<!\d)\d+(?!\d)",x)[1]), data["EDGES"][x]) for x in data["EDGES"].keys() if (re.findall("(?<!\d)\d+(?!\d)",x)[0] <= re.findall("(?<!\d)\d+(?!\d)",x)[1])])
    return nx.floyd_warshall_predecessor_and_distance(graph, weight="DISTANCE")[1]

In [12]:
#data

#read the JSON file
input_content = read_input('Test/Lpr-a-01.json')

#TOLERANCE (values suggested by Garcia-Ayala's paper)
tolerance = (0.1, 0.05)

#DEPOTS
#list of depots
#format: [1, 2, 3]
depots = list()
for p in input_content['DEPOTS']:
    depots.append(int(p))


#CAPACITY
capacity = int(input_content['CAPACITY'])

#VERTICES
#list of vertices
#format: [1, 2, 3, 4]
vertices = list()
for i in input_content['NODES'].keys():
    vertices.append(int(i))

dictionary = dict()

for a in input_content['NODES'].keys():
    for b in input_content['NODES'][a].keys():
        if (int(b),int(a)) not in dictionary.keys():
            dictionary[int(a),int(b)] = int(input_content['NODES'][a][b]['DISTANCE']),int(input_content['NODES'][a][b]['DEMAND'])

#EDGES
#format: (2, 14)
#        (2, 3)
#DISTANCE
#format: {(2, 14): 34, (2, 3): 19}
#DEMAND
#format: {(2, 14): 240, (2, 3): 137}
edges, distance, demand = gp.multidict(dictionary)

#EDGES DELTA
#dictionary of all the edges that have 'i' as one of its ends
#format: {2: [(2, 14), (2, 3), (2, 8)], 14: [(2, 14), (14, 7)]}
edges_delta = dict()

for i in vertices:
    edges_delta[i] = list()
    for e in edges:
        if e[0] == i or e[1] == i:
            edges_delta[i].append(e)

#DEPOT VERTEX
#tuples that relate a depot to a vertex
#format: [(1, 2), (1, 14)]
depot_vertex = list()

for p in depots:
    for i in vertices:
        depot_vertex.append((p, i))

matrix2 = import_floyd_warshall('Test/Lpr-a-01.json')

dictionary = dict()
for p in depots:
    for e in edges:
        dictionary[p,e] = min(matrix2[p][e[0]],matrix2[p][e[1]])


#DEPOT EDGE
#tuples that relate a depot to an edge
#format: ( 1 , (2, 14)  )
#        ( 1 , (2, 3)   )
#DEPOT DISTANCE
#dictionary with the distance between a depot and an edge
#format: {(1, (2, 14)): 60, (1, (2, 3)): 41}
depot_edge, depot_dist = gp.multidict(dictionary)

dictionary = dict()

#PARITY LOOSE
# list of vertices to assign if a vertex loose its parity or not
parity_loose = list()

#list of even parity vertices
vertices_even = list()
#list of odd parity vertices
vertices_odd = list()

for i in range(len(vertices)+1):
    dictionary[i] = 0
    if i != 0:
        parity_loose.append(i)

for e in edges:
    dictionary[e[0]] += 1
    dictionary[e[1]] += 1

for i in range(len(vertices)+1):
    if i != 0 and dictionary[i]%2 == 0:
        vertices_even.append(i)
    elif i != 0 and dictionary[i]%2 == 1:
        vertices_odd.append(i)



#PARITY
parity = list()
parity_0 = list()

for i in vertices:
    for p in depots:
        parity.append((i,p))
        parity_0.append((i,p))


In [ ]:
#declare model
model = gp.Model('RAP')

In [ ]:
#decision variables
b = model.addVars(depot_dist, name="DepotEdgeDistance")
x = model.addVars(depot_edge, vtype=gp.GRB.BINARY, name="DepotEdgeAssign")
w = model.addVars(depot_vertex, vtype=gp.GRB.BINARY, name="DepotVertexIncident")
z = model.addVars(parity, lb = 0, name = "Parity")
z_0 = model.addVars(parity_0, vtype=gp.GRB.BINARY, name = "OddParity")
r = model.addVars(parity_loose, vtype=gp.GRB.BINARY)

In [ ]:
#constrains
constrain_2 = model.addConstrs((gp.quicksum(x[p,e] for p in depots) == 1 for e in edges), name='constrain2')
# constrain_3 = model.addConstrs((gp.quicksum(x[p,s] for s in edges_sigma) - gp.quicksum(x[p,s] for s in S) >= x[p,e] - len(S) for p in depots for e in edges for S in edges), name='constrain 3')
constrain_4 = model.addConstrs((gp.quicksum(x[p,e]*demand[e] for e in edges) <= capacity + tolerance[0] for p in depots), name='constrain4')
constrain_5 = model.addConstrs((gp.quicksum(x[p,e]*demand[e] for e in edges) >= capacity - tolerance[0] for p in depots), name='constrain5')
constrain_6 = model.addConstrs((gp.quicksum(x[p,e] for e in edges_delta[i]) <= 9999*w[p,i] for p in depots for i in vertices), name='constrain6')
constrain_7 = model.addConstrs((gp.quicksum(x[p,e] for e in edges_delta[i]) >= w[p,i] for p in depots for i in vertices), name='constrain7')
constrain_8 = model.addConstrs((gp.quicksum(x[p,e] for e in edges_delta[i]) == 2*z[i,p]+z_0[i,p] for p in depots for i in vertices), name='constrain8')
constrain_9 = model.addConstrs((gp.quicksum(z_0[i,p] for p in depots) >= r[i] for i in vertices_even), name='constrain9')
constrain_10 = model.addConstrs((gp.quicksum(z_0[i,p] for p in depots) <= r[i]*len(depots) for i in vertices_even), name='constrain10')
constrain_11 = model.addConstrs((gp.quicksum(z_0[i,p] for p in depots) - 1 >= r[i] for i in vertices_odd), name='constrain11')
constrain_12 = model.addConstrs((gp.quicksum(z_0[i,p] for p in depots) - 1 <= r[i]*len(depots) for i in vertices_odd), name='constrain12')
constrain_13 = model.addConstr((gp.quicksum(r[i] for i in vertices)/len(vertices) <= tolerance[1]), name='constrain13')


In [ ]:
#objective
model.setObjective(gp.quicksum(b[p,e]*x[p,e] for e in edges for p in depots), GRB.MINIMIZE)

In [ ]:
#save
model.write('RAP.rlp')

In [ ]:
#run
model.optimize()